In [1]:
from utils import (read_video,
                   save_video,
                   measure_distance,
                   draw_player_stats,
                   convert_pixel_distance_to_meters,
                   get_latest_model,
                   cal_stats,
                   pairs_to_points,
                   points_to_pairs,
                   )
import constants
from trackers import PlayerTracker,BallTracker
from court_line_detector import CourtLineDetector, adjust_keypoints_to_lines
from mini_court import MiniCourt
import cv2
import pandas as pd
from copy import deepcopy


In [2]:
# Read Video
input_dir = "input_videos"
input_name = "sample.mp4"
base_name = input_name.split(".")[0]
input_video_path = f"{input_dir}/{input_name}"
video_frames = read_video(input_video_path)

In [3]:
# 检测球员和球
player_tracker = PlayerTracker(model_path='yolov8x')
ball_tracker = BallTracker(model_path='models/last.pt')
# 持久化地读取球员的检测
player_detections = player_tracker.detect_frames(video_frames,
                                                 read_from_stub=True,
                                                 stub_path="tracker_stubs/player_detections.pkl"
                                                 )
# 持久化地读取球的检测                                         
ball_detections = ball_tracker.detect_frames(video_frames,
                                             read_from_stub=True,
                                             stub_path="tracker_stubs/ball_detections.pkl"
                                             )
# 对球进行插值                                         
ball_detections = ball_tracker.interpolate_ball_positions(ball_detections)

In [4]:
# 加载球场线检测模型
court_model_path = "models/keypoints_model_best_4.pth"
# court_model_path = "models/model_tennis_court_det.pt"
court_line_detector = CourtLineDetector(court_model_path)
court_keypoints = court_line_detector.predict(video_frames)

# 选择球员
player_detections = player_tracker.choose_and_filter_players(court_keypoints, player_detections)

In [5]:
print("Player Detections: ", player_detections[0])

Player Detections:  {1: [480.28240966796875, 751.597412109375, 624.837890625, 930.6015014648438], 2: [1030.5684814453125, 201.68313598632812, 1095.9012451171875, 308.51904296875]}


In [6]:
for i in range(0, len(court_keypoints), 2):
    x = int(court_keypoints[i])
    y = int(court_keypoints[i+1])
    print(f'{i//2}: {x}, {y}')

0: 555, 291
1: 1319, 291
2: 284, 834
3: 1590, 839
4: 651, 291
5: 447, 835
6: 1222, 291
7: 1424, 839
8: 622, 373
9: 1254, 374
10: 512, 653
11: 1352, 655
12: 936, 373
13: 930, 654


In [16]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
image = video_frames[10]


# 转换为HSV图像
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 展示图像
cv2.imshow('image', hsv)

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:

# 定义白色的HSV范围
lower_white = np.array([0, 0, 200])
upper_white = np.array([180, 25, 255])

# 根据HSV范围创建一个白色区域的掩码
mask = cv2.inRange(hsv, lower_white, upper_white)

# 寻找白色区域的轮廓
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# 将关键点转换为numpy数组
keypoints = np.array(keypoints)

# 使用DBSCAN进行聚类，找出关键点与线条的靠近关系
db = DBSCAN(eps=20, min_samples=2).fit(keypoints)
labels = db.labels_

# 将关键点移到最近的白色线上
for i, point in enumerate(keypoints):
    label = labels[i]
    if label != -1:
        # 找到同一类的点
        same_cluster_points = keypoints[labels == label]
        # 计算这些点到白色区域的质心的距离
        distances = [cv2.pointPolygonTest(contour, (point[0], point[1]), True) for contour in contours]
        min_distance_index = np.argmin(np.abs(distances))
        closest_contour = contours[min_distance_index]
        # 将点移动到白色线上
        M = cv2.moments(closest_contour)
        if M['m00'] != 0:
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            keypoints[i] = (cx, cy)


In [7]:
new_keypoints = adjust_keypoints_to_lines(video_frames[10], pairs_to_points(court_keypoints))

In [8]:
new_keypoints = points_to_pairs(new_keypoints)

In [9]:
for i in range(0, len(new_keypoints), 2):
    x = int(new_keypoints[i])
    y = int(new_keypoints[i+1])
    print(f'{i//2}: {x}, {y}')

0: 555, 291
1: 1319, 291
2: 284, 834
3: 1590, 839
4: 651, 291
5: 447, 835
6: 1222, 291
7: 1424, 839
8: 622, 373
9: 1254, 374
10: 512, 653
11: 1352, 655
12: 936, 373
13: 930, 654


In [10]:
# 创建迷你球场
mini_court = MiniCourt(video_frames[0])

# 检测击球帧
ball_shot_frames = ball_tracker.get_ball_shot_frames(ball_detections)

# 转换球员和球的位置到迷你球场位置
player_mini_court_detections, ball_mini_court_detections = mini_court.convert_bounding_boxes_to_mini_court_coordinates(
    player_detections, ball_detections, court_keypoints
)

In [11]:
# 统计数据
player_stats_data_df = cal_stats(ball_shot_frames, ball_mini_court_detections, player_mini_court_detections, video_frames, mini_court, constants)

In [12]:
# 绘制输出视频
## 画出球员和球的bbox
output_video_frames= player_tracker.draw_bboxes(video_frames, player_detections)
output_video_frames= ball_tracker.draw_bboxes(output_video_frames, ball_detections)

## 画出球场KPS
output_video_frames  = court_line_detector.draw_keypoints_on_video(output_video_frames)

# 画出迷你球场
output_video_frames = mini_court.draw_mini_court(output_video_frames)
output_video_frames = mini_court.draw_points_on_mini_court(output_video_frames,player_mini_court_detections)
output_video_frames = mini_court.draw_points_on_mini_court(output_video_frames,ball_mini_court_detections, color=(0,255,255))

# 画出球员统计数据
output_video_frames = draw_player_stats(output_video_frames,player_stats_data_df)

## 在左上角画出帧数
for i, frame in enumerate(output_video_frames):
    cv2.putText(frame, f"Frame: {i}",(10,30),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

output_dir = "output_videos"


save_video(output_video_frames, f'{output_dir}/{base_name}_output.avi')
